In [1]:
from nltk.tokenize import word_tokenize

from embedding import E5LargeEmbeddingFunction

import clickhouse_connect
import pandas as pd
import logging

client = clickhouse_connect.get_client(host='y1jzidyt9q.us-east-2.aws.clickhouse.cloud', port=8443, username='default', password='_lQ_JWXYQD3ym')

c:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Данный файл использовался для загрузки данныхв в ClickHouse из собранного и обработанного ранее csv файла

In [2]:
data = pd.read_csv('data_final.csv')
data.shape

(18207, 5)

In [3]:
def get_nltk_chunks(text, chunk_size=200, chunk_overlap=100, language="russian"):
    words = word_tokenize(text, language=language)

    chunks = []

    end_index = chunk_size
    while end_index - chunk_size < len(words):
        sentence = ''
        tokens_counter = 0
        for word in words[end_index - chunk_size:end_index]:
            if word in ',.?!:)»' or (sentence and sentence[-1] in '«('):
                sentence += word
            else:
                sentence += " " + word
            tokens_counter += 1

        if tokens_counter < chunk_size and chunks:
            words = word_tokenize(chunks[-1] + sentence)[-chunk_size:]
            sentence = ''
            for word in words:
                if word in ',.?!:)»' or (sentence and sentence[-1] in '«('):
                    sentence += word
                else:
                    sentence += " " + word
                    
        chunks.append(sentence)
        end_index += chunk_size - chunk_overlap

    return chunks

In [4]:
logging.basicConfig(level=logging.INFO, filename='db.log', filemode="a",
                       format="%(asctime)s %(levelname)s %(message)s")

emb_func = E5LargeEmbeddingFunction()

emb_func.change_mode(new_mode='passage')

In [6]:
for index, row in data[4000:8000].iterrows(): 
    try:
        if index % 25 == 0:
            logging.info(f'{index} done')
        link = row.link
        text = row.text
        pages = row.pages

        if not isinstance(pages, int):
            pages = 0

        is_markdown = row.is_markdown
        is_table = row.is_table
 
        if is_table:
            chunks = get_nltk_chunks(text, chunk_size = 400, chunk_overlap = 100)
        else:
            chunks = get_nltk_chunks(text, chunk_size = 400, chunk_overlap = 100)

        insert_data = []
        for i, chunk in enumerate(chunks):
            insert_data.append((index, i, link, chunk, pages, is_table, *emb_func(chunk)))
            
        client.insert('index_texts_final', insert_data, column_names=['ID', 'chunk_id', 'link', 'text', 'pages', 'is_table', 'embedding'])
    except Exception as e:
        logging.error(index)